### Library and Concatenate

In [ ]:
# Import libraries
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
%matplotlib inline

from matplotlib import rcParams
rcParams['patch.force_edgecolor'] = True
rcParams['patch.facecolor'] = 'b'

In [ ]:
# Concate .csv files
os.chdir("./Sales Data Examples")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension), recursive=True)]
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv.to_csv( "Sales_2019.csv", index=False, encoding='utf-8-sig')

### Read dataframe and Clear outliers

In [ ]:
sales = pd.read_csv("Sales_2019.csv")

In [ ]:
sales.info()

In [ ]:
sales.head()

In [ ]:
sales.describe()

In [ ]:
sales.nunique()

In [ ]:
nan_sales = sales[sales.isna().any(axis=1)]
display(nan_sales.head())

sales = sales.dropna(how='all')
sales = sales[sales['Order Date'].str[0:4] != 'Null']
sales.head()

### Format columns

In [ ]:
sales['Quantity'] = pd.to_numeric(sales['Quantity'])
sales['Price'] = pd.to_numeric(sales['Price'])
sales['Order Date'] = pd.to_datetime(sales['Order Date'], format='%d/%m/%y %H:%M')

In [ ]:
sales['Year'] = pd.DatetimeIndex(sales['Order Date']).year
sales['Month'] = pd.DatetimeIndex(sales['Order Date']).month
sales['Day'] = pd.DatetimeIndex(sales['Order Date']).day
sales['Hour'] = pd.DatetimeIndex(sales['Order Date']).hour
sales['Minute'] = pd.DatetimeIndex(sales['Order Date']).minute
sales.head()

In [ ]:
def district_column(address):
    return address.split(",")[1].split(" ")[2]

sales['District'] = sales['Purchase Address'].apply(lambda x: f"{district_column(x)}")
sales.head()

### EDA

In [ ]:
sales['Sales'] = sales['Quantity'] * sales['Price']

Sales by Month

In [ ]:
df = sales.groupby('Month').agg(Total_Order_Value = ('Sales', 'sum'), Count_Order = ('Order ID', 'count'))

f, ax = plt.subplots(figsize=(9, 6))
plt.title('Sales Trend by Month ($'))
sns.barplot(x=df.index, y="Total_Order_Value", data=df, color="g")
ax2 = plt.twinx()
sns.pointplot(x=df.index, y="Count_Order", data=df, color="b", ax=ax2)

Sales by Hour of the Day

In [ ]:
df = sales.groupby('Hour').agg(Total_Order_Value = ('Sales', 'sum'), Count_Order = ('Order ID', 'count'))

f, ax = plt.subplots(figsize=(9, 6))
plt.title('Sales Trend by Hour ($)')
sns.barplot(x=df.index, y="Total_Order_Value", data=df, color="g")
ax2 = plt.twinx()
sns.pointplot(x=df.index, y="Count_Order", data=df, color="b", ax=ax2)

Sales by Teritory

In [ ]:
df = sales.groupby('District').agg(Total_Order_Value = ('Sales', 'sum'), Count_Order = ('Order ID', 'count'))

f, ax = plt.subplots(figsize=(9, 6))
plt.title('Sales Trend by Teritory ($)')
sns.barplot(x=df.index, y="Total_Order_Value", data=df, color="b")
ax2 = plt.twinx()
sns.pointplot(x=df.index, y="Count_Order", data=df, color="g", ax=ax2)

Sales by Product

In [ ]:
df = sales.groupby('Product').agg(Total_Order_Value = ('Sales', 'sum'), Count_Order = ('Order ID', 'count'))

f, ax = plt.subplots(figsize=(9, 6))
plt.title('Sales trend by Product ($)')
sns.barplot(x="Total_Order_Value", y=df.index, data=df, orient='h', color="b")

Top 5 triple-bundling package

In [ ]:
df = sales[sales['Order ID'].duplicated(keep=False)]
df['Bundle'] = df.groupby('Order ID')['Product'].transform(lambda x: ', '.join(x))
df_pair = df[['Order ID', 'Bundle']].drop_duplicates()
df_pair.head()

In [ ]:
# Referenced: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter

count = Counter()

for row in df_pair['Bundle']:
    row_list = row.split(', ')
    count.update(Counter(combinations(row_list, 3)))

for key, value in count.most_common(5):
    print(key, '-', value, 'times')